# 7章 畳み込みニューラルネットワーク
本章のテーマは、畳み込みニューラルネットワーク(convolutional neural network:CNN)です。CNN は、画像認識や音声認識など、至るところで使わ れています。また、画像認識のコンペティションでは、ディープラーニングによる手 法のほとんどすべてが CNN をベースとしています。本章では、CNN のメカニズム について詳しく説明し、その処理内容を Python で実装していきます。

## 7.1 全体の構造
まずは CNN の大枠を理解するために、CNN のネットワーク構造から見ていくこ とにします。CNN も、これまで見てきたニューラルネットワークと同じで、レゴブ ロックのようにレイヤを組み合わせて作ることができます。ただし CNN の場合、新 たに「Convolution レイヤ(畳み込み層)」と「Pooling レイヤ(プーリング層)」が 登場します。畳み込み層とプーリング層の詳細は次節以降で説明するとして、ここで は、どのようにレイヤを組み合わせて CNN が構築されるかを先に見ていきます。

CNN では、新しく「Convolution レイヤ」と「Pooling レイヤ」が加わります。CNN のレイヤのつながり順は、「Convolution - ReLU - (Pooling)」という流れです(Pooling レイヤは省略されることもあります)。これは、 今までの「Affine - ReLU」というつながりが、「Convolution - ReLU - (Pooling)」 に置き換わったと考えることができます。
CNN で他に注目する点は、出力に近い層では、これまでの「Affine - ReLU」という組み合わせが用いられるということです。また、最後の出力層におい ては、これまでの「Affine - Softmax」の組み合わせが用いられます。これらは一般 的な CNN でよく見られる構成です。

## 7.2 畳み込み層
CNN では、パディング、ストライドなどの CNN 特有の用語が登場します。また、 各層を流れるデータは形状のあるデータ(たとえば、3 次元のデータ)になり、これまでの全結合のネットワークとは異なります。そのため、初めて CNN を学ぶとき は、分かりにくく感じるかもしれません。ここでは、CNN で使われる畳み込み層の 仕組みを、じっくりと時間をかけて見ていきたいと思います。

### 7.2.1 全結合層の問題点
これまで見てきた全結合のニューラルネットワークでは、全結合層(Affine レイ ヤ)を用いました。全結合層では、隣接する層のニューロンがすべて連結されてお り、出力の数は任意に決めることができます。<br>
全結合層の問題点は何でしょうか。それは、データの形状が“無視”されてしまう ことです。たとえば入力データが画像の場合、画像は通常、縦・横・チャンネル方向 の 3 次元の形状です。しかし、全結合層に入力するときには、3 次元のデータを平ら ―― 1 次元のデータ――にする必要があります。実際、これまでの MNIST データ セットを使った例では、入力画像は (1, 28, 28)―― 1 チャンネル、縦 28 ピクセル、 横 28 ピクセル――の形状でしたが、それを 1 列に並べた 784 個のデータを最初の Affine レイヤへ入力しました。<br>

一方、畳み込み層(Convolution レイヤ)は、形状を維持します。画像の場合、入 力データを 3 次元のデータとして受け取り、同じく 3 次元のデータとして、次の層に データを出力します。そのため、CNN では、画像などの形状を有したデータを正し く理解できる(可能性がある)のです。<br>
なお、CNN では、畳み込み層の入出力データを、特徴マップ(feature map)と 言う場合があります。さらに、畳み込み層の入力データを入力特徴マップ(input feature map)、出力データを出力特徴マップ(output feature map)と言います。 本書では、「入出力データ」と「特徴マップ」を同じ意味の言葉として用います。

### 7.2.2 畳み込み演算
畳み込み層で行う処理は「畳み込み演算」です。畳み込み演算は、画像処理で言う ところの「フィルター演算」に相当します。<br>
図7-3 で示すように、畳み込み演算は、入力データに対して、フィルターを適用し ます。この例では、入力データは縦・横方向の形状を持つデータで、フィルターも同 様に、縦・横方向の次元を持ちます。データとフィルターの形状を、(height, width) で表記するとして、この例では、入力サイズは (4, 4)、フィルターサイズは (3, 3)、出 力サイズは (2, 2) になります。なお、文献によっては、ここで述べたような「フィル ター」という用語は、「カーネル」という言葉で表現されることもあります。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.29.22.png">

畳み込み演算は、入力データに対して、フィルターのウィンドウを一定の間隔でス ライドさせながら適用していきます。ここで言うウィンドウとは、図7-4 における灰 色の 3 × 3 の部分を指します。図7-4 に示すように、それぞれの場所で、フィルター の要素と入力の対応する要素を乗算し、その和を求めます(この計算を積和演算と呼 ぶこともあります)。そして、その結果を出力の対応する場所へ格納していきます。 このプロセスをすべての場所で行うことで、畳み込み演算の出力を得ることができ ます。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.34.31.png">

畳み込み演算のバイアス<br>
<img src="./fig/スクリーンショット 2022-08-18 20.27.52.png"><br>
図7-5 に示すように、バイアス項の加算は、フィルター適用後のデータに対して行 われます。ここで示すように、つねにバイアスはひとつ(1 × 1)だけ存在します(こ の例では、フィルター適用後のデータ 4 つに対してバイアスはひとつです)。そのひ とつの値がフィルター適用後のすべての要素に加算されます。

### 7.2.3 パディング
畳み込み層の処理を行う前に、入力データの周囲に固定のデータ(たとえば 0 な ど)を埋めることがあります。これをパディング(padding)と言って、畳み込み演 算ではよく用いられる処理です。たとえば、図 7 - 6 の例では、(4, 4) のサイズの入力 データに対して、幅 1 のパディングを適用しています。幅 1 のパディングとは、周囲 を幅 1 ピクセルの 0 で埋めることを言います。
<img src="./fig/スクリーンショット 2022-08-18 20.45.34.png"><br>
図7-6 に示すように、(4,4) のサイズの入力データはパディングによって、(6,6) の形状になります。そして、(3, 3) のサイズのフィルターをかけると、(4, 4) のサイ ズの出力データが生成されます。この例では、パディングを 1 に設定しましたが、パ ディングの値は 2 や 3 など任意の整数に設定することができます。もし図7-5 の例 でパディングを 2 に設定すれば、入力データのサイズは (8, 8) になり、パディングが 3 であれば、サイズは (10, 10) になります。

パディングを使う主な理由は、出力サイズを調整するためにあります。たとえ ば、(4, 4) のサイズの入力データに (3, 3) のフィルターを適用する場合、出力 サイズは (2, 2) になり、出力サイズは入力サイズから 2 要素分だけ縮小される ことになります。これは、畳み込み演算を何度も繰り返して行うようなディー プなネットワークでは問題になります。なぜなら、畳み込み演算を行うたびに 空間的に縮小されるのであれば、ある時点で出力サイズが 1 になってしまい、 それ以上畳み込み演算を適用できなくなるかもしれません。そのような事態を 回避するには、パディングを使用します。先の例では、パディングの幅を 1 に 設定すれば、入力サイズの (4, 4) に対して、出力サイズも (4, 4) のままサイズ は保たれます。そのため、畳み込み演算によって、空間的なサイズを一定にし たまま次の層へデータを渡すことができるのです。

### 7.2.4 ストライド
フィルターを適用する位置の間隔をストライド(stride)と言います。これまで見 てきた例はすべてストライドが 1 でしたが、たとえば、ストライドを 2 にすると、 図7-7 のように、フィルターを適用する窓の間隔が 2 要素ごとになります。
図 7 - 7 の例では、入力サイズが (7, 7) のデータに対して、ストライドが 2 でフィル ターを適用します。ストライドを 2 に設定することで、出力サイズは (3, 3) になりま す。このように、ストライドは、フィルターを適用する間隔を指定します。
 ここまで見てきたように、ストライドを大きくすると、出力サイズは小さくなりま
す。一方、パディングを大きくすれば、出力サイズは大きくなります。このような関
係性を定式化すると、どうなるでしょうか。続いて、パディングとストライドに対し
て、出力サイズはどのように計算されるのかを見ていきましょう。<br>
<img src="./fig/スクリーンショット 2022-08-19 19.23.45.png">

### 7.2.5 3 次元データの畳み込み演算
これまで見てきた畳み込み演算の例は、縦方向と横方向の 2 次元の形状を対象とし たものでした。しかし、画像の場合、縦・横方向に加えてチャンネル方向も合わせた 3 次元のデータを扱う必要があります。ここでは、先ほどと同じ手順で、チャンネル 方向も合わせた 3 次元データに対して畳み込み演算を行う例を見ていきます。<br>
この例で示すような 3 次元の畳み込み演算で注意する点は、入力データとフィル ターのチャンネル数は同じ値にするということです。この例の場合、入力データと フィルターのチャンネル数はどちらも 3 で一致しています。一方、フィルターのサ イズは好きな値に設定することができます(ただし、チャンネルごとのフィルターの サイズはすべて同じです)。この例ではフィルターのサイズは (3, 3) ですが、それは (2, 2) や (1, 1) または (5, 5) などといったように、好きな値に設定することができま す。しかし、繰り返しになりますが、チャンネル数は入力データのチャンネル数と同 じ値――この例では 3――にしか設定できません。

### 7.2.7 バッチ処理
ニューラルネットワークの処理では、入力データを一束にまとめたバッチ処理を行
いました。これまでの全結合のニューラルネットワークの実装も、バッチ処理に対応
したものであり、これによって、処理の効率化や、学習時のミニバッチへの対応が可
能になりました。<br>
畳み込み演算でも同じように、バッチ処理に対応したいと思います。そのために、各 層を流れるデータは 4 次元のデータとして格納します。具体的には、(batch_num, channel, height, width) という順にデータを格納するものとします。たとえば、 図7-12 の処理を、N 個のデータに対してバッチ処理を行う場合、データの形状は、 次の図7-13 のようになります。
図7-13 のバッチ処理版のデータフローでは、各データの先頭にバッチ用の次元が 追加されています。このように、データは 4 次元の形状として各層を伝わっていきま す。ここでの注意点としては、ネットワークには 4 次元のデータが流れますが、これ は、N 個のデータに対して畳み込み演算が行われている、ということです。つまり、 N 回分の処理を 1 回にまとめて行っているのです。<br>
<img src="./fig/スクリーンショット 2022-08-19 20.13.15.png">
<img src="./fig/スクリーンショット 2022-08-19 20.14.19.png">

### 7.3.1 プーリング層の特徴
#### 学習するパラメータがない
プーリング層は、畳み込み層と違って、学習するパラメータを持ちません。 プーリングは、対象領域から最大値を取る(もしくは平均を取る)だけの処理 なので、学習すべきパラメータは存在しないのです。
#### チャンネル数は変化しない
プーリングの演算によって、入力データと出力データのチャンネル数は変化し ません。図7-15 に示すようにチャンネルごとに独立して計算が行われます。<br>
<br />
<img src="./fig/スクリーンショット 2022-08-19 20.19.43.png"><br>
#### 微小な位置変化に対してロバスト(頑健)
入力データの小さなズレに対して、プーリングは同じような結果を返します。 そのため、入力データの微小なズレに対してロバストです。たとえば、3 × 3 のプーリングの場合、図7-16 に示すように、入力データのズレをプーリング が吸収します(データによって、結果が必ず一致するとはかぎりません)。<br>
<br />
<img src="./fig/スクリーンショット 2022-08-19 20.23.02.png">

[畳み込みネットワークの「基礎の基礎」を理解する　～ディープラーニング入門｜第2回](https://www.imagazine.co.jp/%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E3%80%8C%E5%9F%BA%E7%A4%8E%E3%81%AE%E5%9F%BA%E7%A4%8E%E3%80%8D%E3%82%92%E7%90%86%E8%A7%A3%E3%81%99/)

## 7.4 Convolution / Pooling レイヤの実装

### 7.4.1 4 次元配列
先に説明したとおり、CNN では、各層を流れるデータは 4 次元のデータです。4 次元のデータとは、たとえば、データの形状が (10, 1, 28, 28) だとすると、これは高さ 28・横幅 28 で 1 チャンネルのデータが 10 個ある場合に対応します。これを Python で実装すると、次のようになります。

In [1]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

ここでひとつ目のデータにアクセスするには、単に x[0] と書くだけです(Python のインデックスは 0 から始まることに注意)。同じように 2 つ目のデータは x[1] で アクセスできます。

In [4]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


また、ひとつ目のデータの 1 チャンネル目の空間データにアクセスするには次のよ うに書きます。

In [5]:
x[0, 0]
x[0][0]

array([[6.14628259e-02, 7.09004038e-01, 9.50462849e-02, 1.98307526e-01,
        4.28486333e-01, 5.85891961e-01, 7.11969397e-01, 2.35799676e-01,
        8.78680112e-01, 7.01755968e-01, 6.09343573e-01, 3.10229879e-01,
        6.23609947e-01, 4.75062908e-01, 7.07014532e-01, 4.88343209e-01,
        9.18396838e-01, 6.57976499e-01, 4.09234481e-01, 6.87207285e-01,
        7.25034445e-01, 9.79972674e-01, 4.74113314e-01, 4.21333166e-01,
        5.82010157e-03, 6.78606021e-01, 4.20161191e-01, 1.72708345e-01],
       [9.71411728e-01, 9.07080812e-01, 5.47930231e-01, 8.03979070e-02,
        6.41322035e-02, 6.68487912e-01, 3.22447458e-01, 8.28579233e-01,
        7.58724117e-01, 6.98596542e-01, 4.31738315e-02, 3.10025482e-01,
        6.91917164e-01, 9.38160853e-01, 1.23978567e-01, 5.07677622e-01,
        2.47440751e-01, 5.90435234e-01, 9.00109726e-01, 3.48632006e-02,
        7.12951981e-01, 2.29748427e-01, 8.24340923e-01, 1.27749275e-01,
        8.95318208e-01, 2.56219093e-01, 5.90654473e-01, 6.44439

### 7.4.2 im2col による展開
畳み込み演算の実装は、真面目にやるとすれば、for 文を幾重にも重ねた実装にな るでしょう。そのような実装はやや面倒であり、また、NumPy では for 文を使う と処理が遅くなってしまうという欠点があります(NumPy では、要素アクセスの 際に for 文を使わないことが望まれます)。ここでは、for 文による実装は行わず、 im2col という便利な関数を使ったシンプルな実装を行います。<br>
im2col は、フィルター(重み)にとって都合の良いように入力データを展開する 関数です。図7-17 に示すように、3 次元の入力データに対して im2col を適用する と、2 次元の行列に変換されます(正確には、バッチ数も含めた 4 次元のデータを 2 次元に変換します)。<br>
im2col は、フィルターにとって都合の良いように入力データを展開します。具体 的には、図7-18 に示すように、入力データに対してフィルターを適用する場所の領 域(3 次元のブロック)を横方向に 1 列に展開します。この展開処理を、フィルター を適用するすべての場所で行うのが im2col です。<br>
<img src="./fig/スクリーンショット 2022-08-21 17.22.30.png">

なお、図7-18 の図では、見やすさを優先し、フィルターの適用領域が重ならない ように、ストライドを大きく設定しています。実際の畳み込み演算の場合は、フィ ルター領域が重なる場合がほとんどでしょう。フィルターの適用領域が重なる場合、im2col によって展開すると、展開後の要素の数は元のブロックの要素数よりも多く なります。そのため、im2col を使った実装では通常よりも多くのメモリを消費する という欠点があります。しかし、大きな行列にまとめて計算することは、コンピュー タで計算する上で多くの恩恵があります。たとえば、行列計算のライブラリ(線形代 数ライブラリ)などは、行列の計算実装が高度に最適化されており、大きな行列の掛 け算を高速に行うことができます。そのため、行列の計算に帰着させることで、線形 代数ライブラリを有効に活用することができるのです。

im2col によって入力データを展開してしまえば、その後にやることは、畳み込 み層のフィルター(重み)を 1 列に展開して、2 つの行列の積を計算するだけです
(図 7 - 19 参照)。これは、全結合層の Affine レイヤで行ったこととほとんど同じです。<br>
<img src="./fig/スクリーンショット 2022-08-21 17.26.03.png"><br>
図7-19 に示すように、im2col 方式による出力結果は 2 次元の行列です。CNN の 場合、データは 4 次元配列として格納するので、2 次元の出力データを適切な形状に 整形します。以上が畳み込み層の実装の流れです。

### 7.4.3 Convolution レイヤの実装

In [2]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


import numpy as np

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)


(9, 75)
(90, 75)


ここでは 2 つの例を示しています。ひとつ目は、バッチサイズが 1 で、チャンネル 3 の 7 × 7 のデータ、2 つ目はバッチサイズが 10 で、データの形状はひとつ目と同 じ場合の例です。それぞれ im2col 関数を適用すると、両方のケースで、2 次元目の 要素数は 75 になります。これはフィルター(チャンネル 3、サイズ 5 × 5)の要素 数の総和です。また、バッチサイズが 1 の場合は im2col の結果が (9, 75) のサイ ズです。一方、2 つ目の例はバッチサイズが 10 なので、(90, 75) と 10 倍のデータ が格納されることになります。

In [6]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        # 太字の箇所
        col = im2col(x, FH, FW, self.stride, self.pad) # 入力データをim2colで展開
        col_W = self.W.reshape(FN, -1).T # フィルターをreshapeを使って二次元配列に展開
        out = np.dot(col, col_W) + self.b # 展開した行列の積を計算

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1,2)

        return out

Convolution レイヤの初期化メソッドは、フィルター(重み)とバイアス、ストラ イドとパディングを引数として受け取ります。フィルターは (FN, C, FH, FW) の 4 次元の形状です。なお、FN は Filter Number(フィルターの個数)、C は Channel、 FH は Filter Height、FW は Filter Width を表す略記だとします。
Convolution レイヤの実装では、重要な箇所を太字で示しています。この太字の箇 所では、入力データを im2col で展開し、フィルターも reshape を使って 2 次元配 列に展開します。そして、その展開した行列の積を計算します。<br>
フィルターの展開を行う箇所(コード中の太字)は、図7-19 に示したように、各 フィルターのブロックを 1 列に展開して並べます。ここで、reshape(FN, -1) の ように-1 が指定されていますが、これは、reshape の便利な機能のひとつです。 reshape の際に-1 を指定すると、多次元配列の要素数の辻褄が合うように要素数を まとめてくれるのです。たとえば、(10, 3, 5, 5) の形状の配列は要素数が全部で 750 個ありますが、ここでreshape(10, -1)とすると、(10,75)の形状の配列に整形さ れます。<br>
また、forward の実装では、最後に、出力サイズを適切な形状に整形します。この 整形の際に、NumPy の transpose という関数を使います。transpose は、多次 元配列の軸の順番を入れ替える関数です。図7-20 に示すように、0 から始まるイン デックス(番号)の並びを指定することで、軸の順番を変更します。

### 7.4.4 Pooling レイヤの実装
プーリング層(https://cvml-expertguide.net/terms/dl/layers/pooling-layer/)


In [7]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
    
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        # 最大値 (2)
        out = np.max(col, axis=1)
        # 整形 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out

## 7.5 CNN の実装

● input_dim ―― 入力データの (チャンネル, 高さ, 幅) の次元<br>
● conv_param ―― 畳み込み層のハイパーパラメータ(ディクショナリ)。ディクショナリのキーは下記のとおり<br>
– filter_num ―― フィルターの数<br>
– filter_size ―― フィルターのサイズ<br>
– stride ―― ストライド<br>
– pad ―― パディング<br>
● hidden_size ―― 隠れ層(全結合)のニューロンの数<br>
● output_size ―― 出力層(全結合)のニューロンの数<br>
● weight_init_std ―― 初期化の際の重みの標準偏差<br>

In [1]:
import sys, os
sys.path.append(os.pardir)
from func.layer import *

In [9]:
from multiprocessing import pool
from typing import OrderedDict


class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size =conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        #重みのパラメータの初期化を行う
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])

        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads
        

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from func.trainer import Trainer

本章で学んだこと<br>
● CNN は、これまでの全結合層のネットワークに対して、畳み込み層と プーリング層が新たに加わる。<br>
● 畳み込み層とプーリング層は、im2col(画像を行列に展開する関数)を用 いるとシンプルで効率の良い実装ができる。<br>
● CNN の可視化によって、層が深くなるにつれて高度な情報が抽出されて いく様子が分かる。<br>
● CNN の代表的なネットワークには、LeNet と AlexNet がある。<br>
● ディープラーニングの発展に、ビッグデータと GPU が大きく貢献している。<br>